In [1]:
#載入套件
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import datetime
import selenium
import json
from tqdm import tqdm, trange
import time
import random

In [2]:
#設定DataFrame顯示方式
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format',lambda x:'%4.4f'% x)

In [3]:
#加入反爬元素
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
     }

In [4]:
def getFreeIpProxy():
    """
    取得 Free Proxy List
    """
    free_proxy_urls = [
        f"https://proxylist.geonode.com/api/proxy-list?limit=200&sort_by=lastChecked&sort_type=desc&google=true&protocols=https",
        "https://free-proxy-list.net/"
    ]
    ip_list = []
    for url in tqdm(free_proxy_urls):
        if "geonode" in url:
            res = requests.get(url).json()
            ip_list += [item["ip"]+":"+item["port"] for item in res["data"]]
        elif "free-proxy-list" in url:
            res = requests.get(url)
            ip_list += re.findall('\d+\.\d+\.\d+\.\d+:\d+', res.text)
    return ip_list
    

def getValidIpProxy(num_use):
    """
    透過 ipify 檢驗是否為可用 IP
    """
    
    valid_ips = []
    check_ip_url = "https://api.ipify.org/?format=json"
    ip_list = getFreeIpProxy()

    for ip in tqdm(ip_list):
        try:
            res = requests.get(check_ip_url, proxies={"http": ip, "https": ip}, timeout=5)
            valid_ips.append(ip)
            print(res.json())
        except:
            print("Invalid: ", ip)

        if len(valid_ips) >= num_use:
            break
    
    return valid_ips


In [ ]:
valid_ips = getValidIpProxy(num_use=50)

In [6]:
def changeUserAgentNProxy(maxtimes_changeIp, maxtimes_retry, valid_ips, url, method, payload=None):
    """
    若超過 maxtimes_retry 則換一個 IP 嘗試、若超過 maxtimes_changeIp 次數限制則該資料爬取結果回傳 None
    """
    changedIp = 0
    success = 0
    res = None

    while (changedIp < maxtimes_changeIp) and (success == 0):
        retry = 0 
        timeout = 20
        ip_proxy = random.choice(valid_ips)

        while (retry < maxtimes_retry) and (success == 0):
            try:
                print(ip_proxy, ' Fetching ', url, ' timeout: ', timeout)
                if method == 'get':
                    res = requests.get(url, headers=headers , proxies={'http': ip_proxy, 'https': ip_proxy}, timeout=timeout).content
                elif method == "post":
                    print(payload)
                    res = requests.post(url, data=payload, headers=headers , proxies={'http': ip_proxy, 'https': ip_proxy}, timeout=timeout).content
                else:
                    print("method hasn't defined in function yet.")
                    break
                    
                soup = BeautifulSoup(res, 'html.parser')
                test = soup.find('table')  # 依照頁面不同判斷成功條件不同
                success = 1  # 使跳出 while 迴圈
                print('Success')
            except Exception as e:
                retry += 1  # 當 retry 次數滿會符合更換 proxy 的條件
                timeout += 5  # 每當開始連線失敗就增加 5 秒 timeout 放寬標準
                print('retrying: ', retry)

        changedIp += 1

    return res


In [ ]:
urls = []
for i in range(5):
    page = 'https://www.1111.com.tw/job-bank/job-index.asp?si=2&fs=1&ps=100&page={}'.format(str(i+1))
    resp = requests.get(page, headers=headers)
    nurls = list(set(re.findall('www.1111.com.tw/corp/[0-9]{1,}/', resp.text))) #[0-9]是re套件的正規表達式 {1,}是re套件中匹配n次以上，不重複的意思
    
    # 如果加載新分頁沒有抓到新的資料，就停止迴圈
    if len(urls) == len(list(urls + nurls)):
        break
    urls = list(set(urls + nurls))
    print(len(urls))

In [ ]:
#找500人以上的公司
urls = []
for i in range(150):
    page = 'https://www.1111.com.tw/job-bank/job-index.asp?si=2&fs=1&ps=100&page={}&sf=501*'.format(str(i+1))
    #page = 'https://www.1111.com.tw/job-bank/job-index.asp?si=2&fs=1&ps=100&page={}&sf=101~500'.format(str(i+1))
    resp = requests.get(page, headers=headers)
    nurls = list(set(re.findall('www.1111.com.tw/corp/[0-9]{1,}/', resp.text)))
    
    # 如果加載新分頁沒有抓到新的資料，就停止迴圈
    if len(urls) == len(list(set(urls + nurls))):
        break
    urls = list(set(urls + nurls))
    print(urls)

In [8]:
#處理網址資料
url_b=[]
for i in urls:
    a='http://'+str(i)
    url_b.append(a)
url_b=list(set(url_b))
len(url_b)

789

In [9]:
import time
#測試新的架構
start_time=time.time() #起始執行時間

cmp_name=[]#公司名稱
industry_name=[]#產業類別
cmp_address=[]#公司地址
boss=[]#負責人
cmp_id=[]#統編
cmp_cnt=[]#公司人數
cmp_url=[]#公司網址
capital=[]#資本額
set_time=[]#成立時間
interview=[]#面試須知
cmp_phone=[]#公司電話
cmp_fax=[]#公司傳真
work_cnt=[]#工作機會

#設定防檔參數
maxtimes_changeIp = 5
maxtimes_retry = 3

#迴圈抓不同網址
try:
    for i in tqdm(url_b):
        time.sleep(random.randint(10,15)) #隨機等待
        #init_page = requests.get(i,headers=headers).content
        #加入防檔機制
        init_page = changeUserAgentNProxy(
        maxtimes_changeIp=maxtimes_changeIp, maxtimes_retry=maxtimes_retry, valid_ips=valid_ips, url=i, method="get", payload=None)
        init_soup = BeautifulSoup(init_page, 'lxml') 
        aa = init_soup.find('div', class_="corp_detail")
        #處理公司名稱
        bb=init_soup.find('title')
        bb=bb.getText().split('｜')[0]
        cmp_name.append(bb)
        #處理工作機會 
        try:
            cc = init_soup.find_all('div',{"id": "Job-Detail"})
            for k in cc:
                all_cc_tag = k.find_all('a')
                work_op=all_cc_tag[5].text
                s=[s for s in re.findall(r'-?\d+\.?\d*', work_op)][0]
                work_cnt.append(s)
        except:
            cmp_id.append(np.nan)

        for j in aa.find_all('ul', class_="UI_list_dots"):
            all_li_tag = j.find_all('li')
           #處理公司統編
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "統編" in s]
                cmp_id.append("".join(map(str,matching)))
            except:
                cmp_id.append(np.nan)
            #處理公司位置
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "公司位置" in s]
                tt=t[t.index("".join((map(str,matching))))+2] #返回html腳本中，公司位置後的第二個元素，index是查找list中元素的位置
                cmp_address.append(tt.strip())
            except:
                cmp_address.append(np.nan)
           #處理公司人數
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "公司人數" in s]
                cmp_cnt.append("".join(map(str,matching)))
            except:
                cmp_cnt.append(np.nan)    
            #處理資本額
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "資本額" in s]
                capital.append("".join(map(str,matching)))
            except:
                capital.append(np.nan)
            #處理負責人
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "負責人" in s]
                boss.append("".join(map(str,matching)))
            except:
                boss.append(np.nan)
            #處理成立時間
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "成立時間" in s]
                tt=t[t.index("".join((map(str,matching))))+1] #返回html腳本中，公司位置後的第二個元素，index是查找list中元素的位置
                set_time.append(tt.strip())
            except:
                set_time.append(np.nan)
            #處理公司電話
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "公司電話" in s]
                tt=t[t.index("".join((map(str,matching))))+2]
                cmp_phone.append(tt.strip())
            except:
                cmp_phone.append(np.nan)
            #處理公司傳真
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "公司傳真" in s]
                tt=t[t.index("".join((map(str,matching))))+2]
                cmp_fax.append(tt.strip())
            except:
                cmp_fax.append(np.nan)
            #處理公司網址
            try:
                t=aa.find('ul',class_="UI_list_dots").getText().strip().split("\n")
                matching = [s for s in t if "公司網址" in s]
                cmp_url.append("".join(map(str,matching)))
            except:
                cmp_url.append(np.nan)

            #處理產業類別
            for s in all_li_tag:
                try:
                    if '產業類別' in s.text: #產業類別
                        ins_name=all_li_tag[0].text.strip().split("\n")
                        ins_name=ins_name[-1].strip()
                        industry_name.append(ins_name)
                except:
                    industry_name.append(np.nan)


              

except Exception as err: #例外處理，並印出錯誤訊息
    print(f"{url_b}   網址發生錯誤 ")
    print(err)
    pass
              
end_time=time.time()
print("總執行時間為{total_time}分".format(total_time=(end_time-start_time)/60))

  0%|          | 0/789 [00:00<?, ?it/s]

34.145.226.144:8080  Fetching  http://www.1111.com.tw/corp/2000715/  timeout:  20
retrying:  1
34.145.226.144:8080  Fetching  http://www.1111.com.tw/corp/2000715/  timeout:  25
retrying:  2
34.145.226.144:8080  Fetching  http://www.1111.com.tw/corp/2000715/  timeout:  30
retrying:  3
194.169.167.199:8080  Fetching  http://www.1111.com.tw/corp/2000715/  timeout:  20


  0%|          | 1/789 [01:34<20:45:07, 94.81s/it]

Success
185.58.221.99:8090  Fetching  http://www.1111.com.tw/corp/72428992/  timeout:  20


  0%|          | 2/789 [01:56<11:22:03, 52.00s/it]

Success
54.216.254.207:9000  Fetching  http://www.1111.com.tw/corp/490284/  timeout:  20


  0%|          | 2/789 [02:08<14:00:27, 64.08s/it]

Success
['http://www.1111.com.tw/corp/2000715/', 'http://www.1111.com.tw/corp/72428992/', 'http://www.1111.com.tw/corp/490284/', 'http://www.1111.com.tw/corp/1340005/', 'http://www.1111.com.tw/corp/315695/', 'http://www.1111.com.tw/corp/2130009/', 'http://www.1111.com.tw/corp/890117/', 'http://www.1111.com.tw/corp/1263204/', 'http://www.1111.com.tw/corp/1252393/', 'http://www.1111.com.tw/corp/9720767/', 'http://www.1111.com.tw/corp/72563118/', 'http://www.1111.com.tw/corp/1177287/', 'http://www.1111.com.tw/corp/73480696/', 'http://www.1111.com.tw/corp/1960931/', 'http://www.1111.com.tw/corp/52001268/', 'http://www.1111.com.tw/corp/302400/', 'http://www.1111.com.tw/corp/69649077/', 'http://www.1111.com.tw/corp/2788584/', 'http://www.1111.com.tw/corp/68819835/', 'http://www.1111.com.tw/corp/291585/', 'http://www.1111.com.tw/corp/406773/', 'http://www.1111.com.tw/corp/1903234/', 'http://www.1111.com.tw/corp/68651220/', 'http://www.1111.com.tw/corp/1375084/', 'http://www.1111.com.tw/corp/2

In [10]:
#創建dataframe

dict_1111 = {"cmp_name":cmp_name,
            "industry_name": industry_name,
            "cmp_address": cmp_address,
            "cmp_url": cmp_url,
            "cmp_id": cmp_id,
            "boss":boss,
            "cmp_cnt":cmp_cnt,
            "capital":capital,
            "set_time":set_time,
            "cmp_phone":cmp_phone,
            "cmp_fax":cmp_fax,
            "work_cnt":work_cnt
}
df=pd.DataFrame.from_dict(dict_1111, orient='index') #若不加from_dict會出現array長度的錯誤
df=df.T
print(df.shape)
df.head()


(3, 12)


,cmp_name,industry_name,cmp_address,cmp_url,cmp_id,boss,cmp_cnt,capital,set_time,cmp_phone,cmp_fax,work_cnt
0,台聚管理顧問股份有限公司,其他石油╱煤製品製造,[內湖科技園區],公司網址 : http://www.usig.com.tw,NaN,,,,,,,909
1,三商家購股份有限公司,量販流通相關,台北市中山區民權東路三段4號地下一樓,公司網址 : http://www.simplemart.com.tw/,,,,資本額 : 5億5000萬,,,,None
2,None,None,None,None,,None,None,None,None,None,None,None


In [11]:
#DataFrame資料處理
df['cmp_url']=df.cmp_url.str.split('公司網址 :').str.get(1) #公司網址
df['cmp_cnt']=df.cmp_cnt.str.split(':').str.get(1) #公司人數
df['capital']=df.capital.str.split(':').str.get(1) #資本額
df['etl_date']=datetime.datetime.today()
df.head()

,cmp_name,industry_name,cmp_address,cmp_url,cmp_id,boss,cmp_cnt,capital,set_time,cmp_phone,cmp_fax,work_cnt,etl_date
0,台聚管理顧問股份有限公司,其他石油╱煤製品製造,[內湖科技園區],http://www.usig.com.tw,NaN,,nan,NaN,,,,909,2022-06-19 23:58:42.352466
1,三商家購股份有限公司,量販流通相關,台北市中山區民權東路三段4號地下一樓,http://www.simplemart.com.tw/,,,nan,5億5000萬,,,,None,2022-06-19 23:58:42.352466
2,None,None,None,None,,None,nan,None,None,None,None,None,2022-06-19 23:58:42.352466


In [ ]:
df.shape